In [ ]:
conda create  -f google-cloud-ml.yml
source activate google-cloud-ml


In [ ]:
apt-get install sudo curl and install gcloud

https://cloud.google.com/sdk/docs/quickstart-debian-ubuntu
# Create an environment variable for the correct distribution
export CLOUD_SDK_REPO="cloud-sdk-$(lsb_release -c -s)"

# Add the Cloud SDK distribution URI as a package source
echo "deb http://packages.cloud.google.com/apt $CLOUD_SDK_REPO main" | sudo tee -a /etc/apt/sources.list.d/google-cloud-sdk.list

# Import the Google Cloud Platform public key
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

# Update the package list and install the Cloud SDK
sudo apt-get update && sudo apt-get install google-cloud-sdk



In [ ]:
init gcloud ans set project and authenticate

In [ ]:
gcloud init
cloud auth application-default login
gcloud config set project ai-ml-dl

Recreate the model and save

In [3]:
# -*- coding: utf-8 -*-
import sys
sys.path.insert(0, '../aux/')
from beiras_aux import load_coded_dictionaries, predict_next_chars,load_text
from keras.layers import Dense, Activation, GRU
from keras.models import Sequential
from keras.layers import InputLayer
import os
# Defining the model
def create_gru_model( num_chars):
    """
    Define the network
    :param
        numbers_chars .- Number chars using in the training process
    :return:
        model .- Model network defined
    """
    model = Sequential()
    # 1 Layer .- GRU layer 1 should be an GRU module with 200 hidden units
    model.add(GRU(200, input_shape=(window_size, num_chars), return_sequences=True))
    # 2 Layer .- GRU layer 2 should be an GRU module with 200 hidden units
    model.add(GRU(200))
    # 2 Layer .-  Dense, with number chars unit and softmax activation
    model.add(Dense(num_chars, activation='softmax'))
    return model
import keras.backend as K
K.set_learning_phase(0)
# Input size of the network, the entry text must have the same length
window_size = 100
# Get dictionaries
chars_to_indices, indices_to_chars = load_coded_dictionaries()
number_chars=len(chars_to_indices)
# regenerate the model
model=create_gru_model(number_chars)
model.load_weights('../model_weights/best_beiras_gru_textdata_weights.hdf5')


Using TensorFlow backend.


In [5]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter
import shutil
import os

# Path to export, 1 is the version, 
# we can serve differents version with the same server
export_path = "../export-tf/3"



if os.path.isdir(export_path):
    shutil.rmtree(export_path)
builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'sequence': model.input},
                                  outputs={'scores': model.output})

with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={'serving_default': signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../export-tf/3/saved_model.pb


# Make local prediccions
gcloud ml-engine local predict --model-dir=/home/jota/beiras-rnn/export-tf/3 --json-instances=/home/jota/beiras-rnn/predict/data-ml-local.json > predict/return.json

In [4]:
import json
import numpy as np
y_test=[0]
with open('return.json',"r") as input:
    with open('return2.json',"w") as output:
        n=0;
        for line in input:
            if n==1:
                output.write(line)
            n+=1

with open('return2.json', 'r') as fp:
    y_test=json.load(fp)
test_predict=np.array(y_test)
r = np.argmax(test_predict)  # predict class of each test input
d=indices_to_chars[r]
d

u'm'

# Deploy model

In [ ]:
Make bucket and create model
MODEL_NAME=BeirasRnn
gcloud ml-engine models create $MODEL_NAME --enable-logging
gsutil mb gs://beiras_rnn

In [ ]:
gsutil cp -r ./export-tf/3 gs://beiras_rnn/export-google-ml/10
MODEL_NAME=BeirasRnn
DEPLOYMENT_SOURCE="gs://beiras_rnn/export-google-ml/10"
gcloud ml-engine versions create "v10" --model $MODEL_NAME  --origin $DEPLOYMENT_SOURCE


pip install --upgrade google-api-python-client

In [6]:
import googleapiclient.discovery
import sys
import numpy as np
sys.path.insert(0, '../aux/')
from beiras_aux import load_coded_dictionaries, predict_next_chars, clean_text


input_init="se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu"
#input_init="pla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran"
# Load values
window_size = 100
chars_to_indices, indices_to_chars = load_coded_dictionaries()
number_chars=len(chars_to_indices)
# Clean the text
input_clean=clean_text(input_init.lower())
input_clean = input_clean[:window_size]
# Text to array [1,input_lenght,num_chars]
x_test = np.zeros((window_size, number_chars))
for t, char in enumerate(input_clean):
    if char in chars_to_indices:
        x_test[ t, chars_to_indices[char]] = 1.
#x_test   
#x_test = np.zeros((number_chars,window_size))
#for t, char in enumerate(input_clean):
#    if char in chars_to_indices:
#        x_test[ chars_to_indices[char],t] = 1.
print(len(x_test[0]))    
project="ai-ml-dl"
model="BeirasRnn"
version="v10"
service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}'.format(project, model)
if version is not None:
        name += '/versions/{}'.format(version)
instances={'sequence':x_test.tolist()}
response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()
if 'error' in response:
    raise RuntimeError(response['error'])
test_predict=np.array(response['predictions'][0]['scores'])
r = np.argmax(test_predict)  # predict class of each test input
print(test_predict)
print(r,indices_to_chars[r])

/Users/jotavaladouro/anaconda/envs/google-cloud-ml/lib/python2.7/site-packages/ipykernel_launcher.py:20: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


55
[  3.51546885e-04   1.14319491e-11   2.63947986e-05   1.84414816e-07
   3.36886907e-04   2.11999195e-05   2.33439205e-05   1.53388009e-06
   8.02952115e-07   3.70742782e-06   1.38431062e-06   5.31530422e-05
   1.34768698e-06   1.37859752e-05   3.21813583e-01   2.69034554e-05
   2.03325999e-05   2.56823409e-06   4.12520580e-02   9.74724799e-07
   3.45889275e-05   3.13729830e-02   2.14483138e-04   3.60666263e-06
   1.53559540e-06   2.02687289e-07   6.53400889e-07   7.27140832e-06
   5.08609153e-02   2.23578081e-06   1.06179132e-05   8.22419770e-06
   1.15277899e-05   6.45663531e-05   5.44817686e-01   2.26951920e-06
   9.90975650e-07   9.07722006e-06   7.59744034e-06   6.03258854e-07
   1.23464641e-11   1.07990136e-11   1.55533117e-03   1.19396784e-11
   2.84433155e-03   1.18369602e-11   2.25673048e-05   1.17487201e-11
   4.03751437e-06   1.10910300e-11   3.98885738e-03   1.20199024e-11
   2.01249553e-04   1.92584679e-07   1.13429536e-07]
(34, u'u')


# Predict sentece

In [7]:
# Define a function
def predict_one(text_predict,service,model_name,window_size,chars_to_indices, indices_to_chars):
    # Convert input sequence to array
    number_chars=len(chars_to_indices)
    x_test = np.zeros((window_size,number_chars))
    for t, char in enumerate(text_predict):
       x_test[t,chars_to_indices[char]] = 1.
    #print(x_test.shape)
    x_test=x_test[:window_size,:]
    #Prepare the request
    instances={'sequence':x_test.tolist()}
    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()
    if 'error' in response:
        raise RuntimeError(response['error'])
    test_predict=np.array(response['predictions'][0]['scores'])
    r = np.argmax(test_predict)  # predict class of each test input
    return (indices_to_chars[r])

In [8]:
# Complete a sequence using the server
def predict_window(text_predict,number_predict,window_size,lproject,lmodel,lversion):
    
    # Get dictionaries
    chars_to_indices, indices_to_chars = load_coded_dictionaries()
    # Clean the test
    input_clean=clean_text(text_predict.lower())
    # Get stub
    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(lproject, lmodel)
    if lversion is not None:
        name += '/versions/{}'.format(lversion)
    print(name)
    # Call server for all charazters
    for i in range(number_predict):
        d=predict_one(input_clean[i:],service,name,window_size,chars_to_indices, indices_to_chars)
        input_clean+=d
    return input_clean

In [10]:
import googleapiclient.discovery
project="ai-ml-dl"
model="BeirasRnn"
version="v10"

input_init=u"se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu"
#input_init="pla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran"

window_size = 100
number_predict=200
      

response=predict_window(input_init[:window_size],number_predict,
                        window_size,project,model,version)
print(response)

projects/ai-ml-dl/models/BeirasRnn/versions/v10
se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcumalo de estado español e o proceso de descomposición do poder constitucional e desembocar nestas asembleas a unha concepción do sistema de estado e o proceso de descomposición do sistema-mundo as cond
